<a href="https://colab.research.google.com/github/DaichiFutamata/DataDrivenFinance/blob/main/bitbank_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install git+https://github.com/bitbankinc/python-bitbankcc@e82d8e59a7f5fe43339deacc605a532749ef9859\#egg=python-bitbankcc

  Cloning https://github.com/bitbankinc/python-bitbankcc (to revision e82d8e59a7f5fe43339deacc605a532749ef9859) to /tmp/pip-install-wkufj9e4/python-bitbankcc_f5bcdc5ac08c44899a2525df658e9cb9
  Running command git clone --filter=blob:none --quiet https://github.com/bitbankinc/python-bitbankcc /tmp/pip-install-wkufj9e4/python-bitbankcc_f5bcdc5ac08c44899a2525df658e9cb9
  Running command git rev-parse -q --verify 'sha^e82d8e59a7f5fe43339deacc605a532749ef9859'
  Running command git fetch -q https://github.com/bitbankinc/python-bitbankcc e82d8e59a7f5fe43339deacc605a532749ef9859
  Running command git checkout -q e82d8e59a7f5fe43339deacc605a532749ef9859
  Resolved https://github.com/bitbankinc/python-bitbankcc to commit e82d8e59a7f5fe43339deacc605a532749ef9859
  Preparing metadata (setup.py) ... done
  Created wheel for python-bitbankcc: filename=python_bitbankcc-0.0.4-py3-none-any.whl size=9220 sha256=5a1e93ad9954384bda802070a716742f98657db59632fbb8f8d963c4d45da140
  Stored in directory: /roo

引用元→https://modernimal.com/crypto-how-to-use-bitbank-api-5677

In [1]:
from datetime import datetime, timedelta

start_date = datetime.strptime("20240216", "%Y%m%d")
end_date = datetime.strptime("20250216", "%Y%m%d")

date_list = []
current_date = start_date
while current_date <= end_date:
    date_list.append(current_date.strftime("%Y%m%d"))
    current_date += timedelta(days=1)

# リストをstr型に変換
date_str = ", ".join(date_list)

# 結果を表示（最初の一部と最後の一部）
print(date_str[:50] + "...")  # 最初の50文字
print(date_str[-50:])  # 最後の50文字


20240216, 20240217, 20240218, 20240219, 20240220, ...
, 20250212, 20250213, 20250214, 20250215, 20250216


In [35]:
import python_bitbankcc
pub = python_bitbankcc.public()
value = pub.get_ticker('btc_jpy')
pri = python_bitbankcc.private('d0e1d428-dcb5-48fe-9dab-bbea713dd324','d2a7b14cc47da43cfd37cef73438da81c3901fc49dd51c522e37b36087e4bf82')
value2 = pub.get_ticker('eth_jpy')
#2024年から一年分の日付をYYMMdd型で取得
from datetime import datetime, timedelta

start_date = datetime.strptime("20240216", "%Y%m%d")
end_date = datetime.strptime("20250216", "%Y%m%d")

date_list = []
current_date = start_date
while current_date <= end_date:
    date_list.append(current_date.strftime("%Y%m%d"))
    current_date += timedelta(days=1)
#date_listに格納されているか確認
for date in date_list:
  print("リストの値{0}".format(date))
  if date =='20250216':
    print('終了')
    break
#2024年から2025年までのイーサリアムのローソク足情報を取得したい
import pandas as pd
import python_bitbankcc
pub = python_bitbankcc.public()
value = pub.get_ticker('btc_jpy')
pri = python_bitbankcc.private('d0e1d428-dcb5-48fe-9dab-bbea713dd324','d2a7b14cc47da43cfd37cef73438da81c3901fc49dd51c522e37b36087e4bf82')
value2 = pub.get_ticker('eth_jpy')
#2024年から一年分の日付をYYMMdd型で取得
from datetime import datetime, timedelta

start_date = datetime.strptime("20240216", "%Y%m%d")
end_date = datetime.strptime("20250216", "%Y%m%d")

date_list = []
current_date = start_date
while current_date <= end_date:
    date_list.append(current_date.strftime("%Y%m%d"))
    current_date += timedelta(days=1)
#date_listに格納されているか確認
for date in date_list:
  print("リストの値{0}".format(date))
  if date =='20250216':
    print('終了')
    break
#リアルタイムデータの取得
import python_bitbankcc
pub = python_bitbankcc.public()
import pandas as pd
import numpy as np
int_date = [str(date)for date in date_list]
Real_time = pub.get_depth('eth_jpy')
#↓こんな感じに選べます
import python_bitbankcc
pub  = python_bitbankcc.public()
Real_time = pub.get_depth('eth_jpy')
print(Real_time['asks'])
print(Real_time['bids'])
#これ結局get_depthって最新価格の事なんかな
#最新価格というより売りと買いの板情報みたいやな
#説明書get_tickerで['last']指定したら最新価格が取得できるみたい
Real_time = pub.get_ticker('eth_jpy')
print(Real_time['last'])#←やっとやりたかった事ができた
#次は連続してこの最新価格をlistに格納するプログラムを書きたい
#＋よくあるエラーについて
import python_bitbankcc
pub = python_bitbankcc.public #←に()つけ忘れただけで動作しなくなるので注意
Real_price = pub.get_ticker('eth_jpy')

[['429985', '2.7759'], ['429990', '0.0276'], ['429991', '0.4999'], ['430000', '2.3020'], ['430028', '0.0184'], ['430080', '0.9170'], ['430081', '0.3000'], ['430101', '0.1520'], ['430114', '81.6037'], ['430231', '0.0605'], ['430233', '2.7789'], ['430292', '0.0632'], ['430296', '0.1212'], ['430351', '7.7659'], ['430352', '0.1653'], ['430373', '0.1066'], ['430746', '0.1000'], ['430747', '0.1000'], ['430750', '3.9564'], ['430757', '1.5000'], ['430937', '3.0000'], ['430956', '0.0040'], ['431977', '0.1600'], ['431978', '1.6000'], ['432499', '0.0500'], ['432500', '0.0028'], ['432519', '1.2000'], ['432774', '0.0100'], ['432900', '0.0200'], ['432902', '0.0510'], ['433000', '0.0500'], ['433180', '0.0472'], ['433500', '0.0122'], ['433569', '4.7000'], ['433584', '8.5666'], ['434193', '0.0020'], ['434998', '0.1000'], ['435000', '3.0294'], ['435114', '0.0001'], ['435500', '0.0200'], ['435713', '0.0010'], ['435817', '0.0472'], ['436145', '0.0006'], ['436193', '0.0020'], ['437000', '0.6873'], ['437187

In [44]:
#1000回最新価格を取得するプログラム
for Real_price_dict in range(1000):
  import python_bitbankcc
  pub = python_bitbankcc.public()
  Real_price = pub.get_ticker('eth_jpy')
  print(Real_price['last'])

427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974
427974

KeyboardInterrupt: 

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
#Talibのインストール
#Cライブラリの整備
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xvzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
%cd ..
#Talibのホイールを直接インストール
!pip install ta-lib-bin
import talib
print(talib.get_functions())

--2025-02-14 05:40:09--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 104.18.12.149, 104.18.13.149, 2606:4700::6812:c95, ...
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|104.18.12.149|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2025-02-14 05:40:09--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 104.18.12.149, 104.18.13.149, 2606:4700::6812:d95, ...
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: http://netactuate.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz?viasf=1 [following]
--2025-02-14 05:40:09--  http://netactuate.dl.

In [ ]:
#ccxtのインストール
!pip install ccxt
import ccxt

In [ ]:
# @title シンプルなトレードbot例
import time
import pandas as pd
import ccxt
import talib as ta

exchange = ccxt.coincheck()
exchange.apiKey = 'あなたのAPIキー'
exchange.secret = 'あなたのシークレットキー'

crypto = 'BTC'          # 取引する仮想通貨
currency = 'JPY'        # 日本円建て
interval = 60*5         # チャート足（60*5なら 5分足）
duration = 20           # 移動平均のサイズ
trading_amount = 30000  # 一度の売買で使う金額（単位=currency）

symbol = crypto + '/' + currency
position = 0            # 自動売買中に持っているポジション
last_rate = 0           # 最後に約定したレート（単位=currency）

df = pd.DataFrame()

while True:
  time.sleep(interval)

  ticker_info = exchange.fetch_ticker(symbol)
  df = pd.concat([df, pd.DataFrame({'price': [ticker_info['last']]})], ignore_index=True)

  if len(df) < duration:
    continue

  df['upper'], df['middle'], df['lower'] = ta.BBANDS(df['price'], timeperiod=duration, nbdevup=2, nbdevdn=2, matype=0)

  if position:
    if df['price'].iloc[-1] > df['upper'].iloc[-1] and last_rate < df['price'].iloc[-1]:
      exchange.create_order(symbol=symbol, type='market', side='sell', amount=position)
      print('sell ' + str(position) + crypto + ' @' + str(ticker_info['last']))
      position = 0

  else:
    ticker_balance = exchange.fetch_balance()
    if float(ticker_balance[currency]['free']) > trading_amount and df['price'].iloc[-1] < df['lower'].iloc[-1]:
      amount = round(trading_amount / float(ticker_info['last']), 5)
      exchange.create_order(symbol=symbol, type='market', side='buy', amount=amount)
      print('buy ' + str(amount) + crypto + ' @' + str(ticker_info['last']))
      position = amount
      last_rate = ticker_info['last']

  df = df.iloc[1:, :]

KeyboardInterrupt: 

↑引用元　https://modernimal.com/learn-how-to-use-ccxt-by-python-crypto-bot-6231#ccxtbot